<a href="https://colab.research.google.com/github/udlbook/udlbook/blob/main/Notebooks/Chap10/10_2_Convolution_for_MNIST_1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 10.2: Convolution for MNIST-1D**

This notebook investigates a 1D convolutional network for MNIST-1D as in figure 10.7 and 10.8a.

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.



In [1]:
# Run this if you're in a Colab to install MNIST 1D repository
!pip install git+https://github.com/greydanus/mnist1d

  Cloning https://github.com/greydanus/mnist1d to c:\users\georg\appdata\local\temp\pip-req-build-o0d_cmok
  Resolved https://github.com/greydanus/mnist1d to commit 7878d96082abd200c546a07a4101fa90b30fdf7e
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for mnist1d: filename=mnist1d-0.0.2.post16-py3-none-any.whl size=14720 sha256=fe9445e0a3ec165c15402dc2cce91425ee103494db101de6dd14ea8cf659a6c9
  Stored in directory: C:\Users\georg\AppData\Local\Temp\pip-ephem-wheel-cache-ybof8m3u\wheels\c4\34\e0\a3b4376888d7486638e921a69788a6309c58168eb2b2183b5b
Successfully built mnist1d
  Attempting uninstall: mnist1d
    Found existing installation: mnist1d 0.0.2.post15
    Uninstalling mnist1d-0.0.2.post15

  Running command git clone --filter=blob:none --quiet https://github.com/greydanus/mnist1d 'C:\Users\georg\AppData\Local\Temp\pip-req-build-o0d_cmok'


In [2]:
import numpy as np
import os
import torch, torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import mnist1d
import random

In [3]:
args = mnist1d.data.get_dataset_args()
data = mnist1d.data.get_dataset(args, path='./mnist1d_data.pkl', download=False, regenerate=False)

# The training and test input and outputs are in
# data['x'], data['y'], data['x_test'], and data['y_test']
print("Examples in training set: {}".format(len(data['y'])))
print("Examples in test set: {}".format(len(data['y_test'])))
print("Length of each example: {}".format(data['x'].shape[-1]))

Successfully loaded data from ./mnist1d_data.pkl
Examples in training set: 4000
Examples in test set: 1000
Length of each example: 40


In [4]:
# Load in the data
train_data_x = data['x'].transpose()
train_data_y = data['y']
val_data_x = data['x_test'].transpose()
val_data_y = data['y_test']
# Print out sizes
print("Train data: %d examples (columns), each of which has %d dimensions (rows)"%((train_data_x.shape[1],train_data_x.shape[0])))
print("Validation data: %d examples (columns), each of which has %d dimensions (rows)"%((val_data_x.shape[1],val_data_x.shape[0])))

Train data: 4000 examples (columns), each of which has 40 dimensions (rows)
Validation data: 1000 examples (columns), each of which has 40 dimensions (rows)


Define the network

In [5]:
# There are 40 input dimensions and 10 output dimensions for this data
# The inputs correspond to the 40 offsets in the MNIST1D template.
D_i = 40
# The outputs correspond to the 10 digits
D_o = 10


# TODO Create a model with the following layers
# 1. Convolutional layer, (input=length 40 and 1 channel, kernel size 3, stride 2, padding="valid", 15 output channels )
# 2. ReLU
# 3. Convolutional layer, (input=length 19 and 15 channels, kernel size 3, stride 2, padding="valid", 15 output channels )
# 4. ReLU
# 5. Convolutional layer, (input=length 9 and 15 channels, kernel size 3, stride 2, padding="valid", 15 output channels)
# 6. ReLU
# 7. Flatten (converts 4x15) to length 60
# 8. Linear layer (input size = 60, output size = 10)
# References:
# https://pytorch.org/docs/1.13/generated/torch.nn.Conv1d.html?highlight=conv1d#torch.nn.Conv1d
# https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html
# https://pytorch.org/docs/1.13/generated/torch.nn.Linear.html?highlight=linear#torch.nn.Linear

# NOTE THAT THE CONVOLUTIONAL LAYERS NEED TO TAKE THE NUMBER OF INPUT CHANNELS AS A PARAMETER
# AND NOT THE INPUT SIZE.

# Replace the following function:
model = nn.Sequential(
nn.Conv1d(1, 15, 3, 2, padding="valid"),
nn.ReLU(),
nn.Conv1d(15, 15, 3, 2, padding="valid"),
nn.ReLU(),
nn.Conv1d(15, 15, 3, 2, padding="valid"),
nn.ReLU(),
nn.Flatten(),
nn.Linear(60, 10)
)

#####
'''
n.Flatten(),
nn.Linear(40, 100),
nn.ReLU(),
nn.Linear(100, 100),
nn.ReLU(),
nn.Linear(100, 10))
'''




'\nn.Flatten(),\nnn.Linear(40, 100),\nnn.ReLU(),\nnn.Linear(100, 100),\nnn.ReLU(),\nnn.Linear(100, 10))\n'

In [6]:
# He initialization of weights
def weights_init(layer_in):
  if isinstance(layer_in, nn.Linear):
    nn.init.kaiming_uniform_(layer_in.weight)
    layer_in.bias.data.fill_(0.0)

In [ ]:
# choose cross entropy loss function (equation 5.24 in the loss notes)
loss_function = nn.CrossEntropyLoss()
# construct SGD optimizer and initialize learning rate and momentum
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05, momentum=0.9)
# object that decreases learning rate by half every 20 epochs
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
# create 100 dummy data points and store in data loader class
x_train = torch.tensor(train_data_x.transpose().astype('float32'))
y_train = torch.tensor(train_data_y.astype('long')).long()
x_val= torch.tensor(val_data_x.transpose().astype('float32'))
y_val = torch.tensor(val_data_y.astype('long')).long()

# load the data into a class that creates the batches
data_loader = DataLoader(TensorDataset(x_train,y_train), batch_size=100, shuffle=True, worker_init_fn=np.random.seed(1))

# Initialize model weights
model.apply(weights_init)

# loop over the dataset n_epoch times
n_epoch = 100
# store the loss and the % correct at each epoch
losses_train = np.zeros((n_epoch))
errors_train = np.zeros((n_epoch))
losses_val = np.zeros((n_epoch))
errors_val = np.zeros((n_epoch))

for epoch in range(n_epoch):
  # loop over batches
  for i, data in enumerate(data_loader):
    # retrieve inputs and labels for this batch
    x_batch, y_batch = data
    # zero the parameter gradients
    optimizer.zero_grad()
    # forward pass -- calculate model output
    pred = model(x_batch[:,None,:])
    # compute the loss
    loss = loss_function(pred, y_batch)
    # backward pass
    loss.backward()
    # SGD update
    optimizer.step()

  # Run whole dataset to get statistics -- normally wouldn't do this
  pred_train = model(x_train[:,None,:])
  pred_val = model(x_val[:,None,:])
  _, predicted_train_class = torch.max(pred_train.data, 1)
  _, predicted_val_class = torch.max(pred_val.data, 1)
  errors_train[epoch] = 100 - 100 * (predicted_train_class == y_train).float().sum() / len(y_train)
  errors_val[epoch]= 100 - 100 * (predicted_val_class == y_val).float().sum() / len(y_val)
  losses_train[epoch] = loss_function(pred_train, y_train).item()
  losses_val[epoch]= loss_function(pred_val, y_val).item()
  print(f'Epoch {epoch:5d}, train loss {losses_train[epoch]:.6f}, train error {errors_train[epoch]:3.2f},  val loss {losses_val[epoch]:.6f}, percent error {errors_val[epoch]:3.2f}')

  # tell scheduler to consider updating learning rate
  scheduler.step()

# Plot the results
fig, ax = plt.subplots()
ax.plot(errors_train,'r-',label='train')
ax.plot(errors_val,'b-',label='validation')
ax.set_ylim(0,100); ax.set_xlim(0,n_epoch)
ax.set_xlabel('Epoch'); ax.set_ylabel('Error')
ax.set_title('Part I: Validation Result %3.2f'%(errors_val[-1]))
ax.legend()
plt.show()

Epoch     0, train loss 1.915151, train error 75.75,  val loss 1.915648, percent error 78.00
Epoch     1, train loss 1.556637, train error 61.05,  val loss 1.573053, percent error 63.00
Epoch     2, train loss 1.394205, train error 56.17,  val loss 1.419114, percent error 58.70
Epoch     3, train loss 1.282007, train error 51.25,  val loss 1.312380, percent error 54.40
Epoch     4, train loss 1.283606, train error 52.80,  val loss 1.313188, percent error 54.70
Epoch     5, train loss 1.146468, train error 46.05,  val loss 1.171881, percent error 48.20
Epoch     6, train loss 1.056706, train error 40.88,  val loss 1.092210, percent error 45.70
Epoch     7, train loss 1.027330, train error 43.00,  val loss 1.059896, percent error 45.20
Epoch     8, train loss 0.975439, train error 37.80,  val loss 1.011278, percent error 40.60
Epoch     9, train loss 0.829387, train error 33.50,  val loss 0.881838, percent error 36.20
Epoch    10, train loss 0.705416, train error 26.90,  val loss 0.75561

Epoch    90, train loss 0.026311, train error 0.45,  val loss 0.394473, percent error 8.70
Epoch    91, train loss 0.026065, train error 0.43,  val loss 0.398289, percent error 8.80
Epoch    92, train loss 0.025859, train error 0.50,  val loss 0.395829, percent error 8.80
Epoch    93, train loss 0.025797, train error 0.45,  val loss 0.396433, percent error 8.80
Epoch    94, train loss 0.025661, train error 0.45,  val loss 0.399528, percent error 8.80
Epoch    95, train loss 0.025449, train error 0.43,  val loss 0.399466, percent error 9.10
Epoch    96, train loss 0.025529, train error 0.43,  val loss 0.402205, percent error 8.60
Epoch    97, train loss 0.025256, train error 0.43,  val loss 0.400952, percent error 8.80
Epoch    98, train loss 0.025221, train error 0.50,  val loss 0.399087, percent error 8.60
Epoch    99, train loss 0.024823, train error 0.45,  val loss 0.401029, percent error 8.80
